In [8]:
import pandas as pd
df=pd.read_csv('Clean_Constraint_English.csv')
df

,id,tweet,label,clean_tweet,label_encoded
0,1,The CDC currently reports 99031 deaths. In gen...,real,cdc currently report death general discrepancy...,1
1,2,States reported 1121 deaths a small rise from ...,real,state reported death small rise last tuesday s...,1
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,politically correct woman almost us pandemic e...,0
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,testing laboratory india th august test done dg,1
4,5,Populous states can generate large case counts...,real,populous state generate large case count look ...,1
...,...,...,...,...,...
10695,2136,Donald Trump wrongly claimed that New Zealand ...,fake,donald trump wrongly claimed new zealand big s...,0
10696,2137,Current understanding is #COVID19 spreads most...,real,current understanding spread mostly person per...,1
10697,2138,Nothing screams “I am sat around doing fuck al...,fake,nothing scream sat around fuck lockdown quite ...,0
10698,2139,Birx says COVID-19 outbreak not under control ...,fake,birx say covid outbreak control people move,0


In [9]:
df1_clean=df

In [10]:
# Supprimer les lignes avec des valeurs NaN dans 'clean_tweet'
df1_clean = df1_clean.dropna(subset=['clean_tweet'])
#supression des colonnes id,tweet et label
df1_clean= df1_clean.drop(['id', 'tweet', 'label'], axis=1)
df1_clean

,clean_tweet,label_encoded
0,cdc currently report death general discrepancy...,1
1,state reported death small rise last tuesday s...,1
2,politically correct woman almost us pandemic e...,0
3,testing laboratory india th august test done dg,1
4,populous state generate large case count look ...,1
...,...,...
10695,donald trump wrongly claimed new zealand big s...,0
10696,current understanding spread mostly person per...,1
10697,nothing scream sat around fuck lockdown quite ...,0
10698,birx say covid outbreak control people move,0


In [11]:
from sklearn.model_selection import train_test_split
# Séparer les features (X) et labels (y)
X = df1_clean['clean_tweet']
y = df1_clean['label_encoded']

# Division en train (80%) et test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Vérifier la répartition des classes
y_train.value_counts(), y_test.value_counts()


(label_encoded
 1    4479
 0    4078
 Name: count, dtype: int64,
 label_encoded
 1    1120
 0    1020
 Name: count, dtype: int64)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score

# Vectorisation TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Limite à 5000 features pour éviter trop de dimensions
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Définition du modèle XGBoost
xgb_model = xgb.XGBClassifier(eval_metric="logloss")

# Définition des hyperparamètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],  # Nombre d'arbres
    'max_depth': [3, 6, 9],  # Profondeur maximale des arbres
    'learning_rate': [0.01, 0.1, 0.3]  # Taux d'apprentissage
}

# Validation croisée avec 5 folds
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring="accuracy",
    cv=cv,
    n_jobs=-1,  # Utilisation de tous les cœurs CPU pour accélérer # Remplacer par n_jobs=1 si le PC est lent
    verbose=1
)

# Entraînement avec recherche des meilleurs paramètres
grid_search.fit(X_train_tfidf, y_train)

# Affichage des meilleurs paramètres trouvés
print(f"Meilleurs paramètres : {grid_search.best_params_}")

# Réentraînement du modèle avec les meilleurs paramètres trouvés
best_xgb = grid_search.best_estimator_

# Prédiction sur le test set
y_pred_xgb = best_xgb.predict(X_test_tfidf)

# Calcul de l'accuracy finale
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
print(f"Accuracy sur le test set : {xgb_accuracy:.4f}")


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Meilleurs paramètres : {'learning_rate': 0.3, 'max_depth': 6, 'n_estimators': 300}
Accuracy sur le test set : 0.9117
